# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Load, Engineer & Connect</span>

<span style="font-width:bold; font-size: 1.4rem;"> This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, we will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for batch data with a goal of training and deploying a model that can predict fraudulent transactions.</span>

## **🗒️ This notebook is divided in 4 sections:** 
1. Loading the data and do feature engineeing,
2. Connect to the Hopsworks feature store,
3. Create feature groups and upload them to the feature store.
4. Explore feature groups from the UI.

![tutorial-flow](../../images/01_featuregroups.png)

First of all we will load the data and do some feature engineering on it.

### 📝 Import librararies 

In [1]:
import datetime
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from tqdm import tqdm

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

The data we will use comes from three different CSV files:

- `transactions.csv`: transaction information such as timestamp, location, and the amount. 
- `alert_transactions.csv`: Suspicious Activity Report (SAR) transactions.
- `party.csv`: User profile information.

In a production system, these CSV files would originate from separate data sources or tables, and probably separate data pipelines. **All three files have a customer id column `id` in common, which we can use for joins.**

Let's go ahead and load the data.

#### ⛳️ Transactions dataset

In [ ]:
transactions_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/aml/transactions.csv", parse_dates = ['tran_timestamp'])
transactions_df.head(5)

#### ⛳️ Alert Transactions dataset

In [ ]:
alert_transactions = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/aml/alert_transactions.csv")
alert_transactions.head()

#### ⛳️ Party dataset

In [ ]:
party = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/aml/party.csv")
party.head()

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

#### To investigate patterns of suspicious activities you will make time window aggregates such monthly frequency, total, mean and standard deviation of amount of incoming and outgoing transasactions.  


In [ ]:
# rename some columns
transactions_df = transactions_df.rename(columns={"src": "source",
                                                  "dst": "target"}, errors="raise")

# select interested columns
transactions_df = transactions_df[["source", "target", "tran_timestamp", "tran_id", "base_amt"]]
transactions_df.head()

##### Outgoing transactions

In [ ]:
out_df = transactions_df.groupby([pd.Grouper(key='tran_timestamp', freq='M'), 'source'])\
                            .agg(monthly_count=('source','count'), 
                                 monthly_total_amount=('base_amt','sum'),
                                 monthly_mean_amount=('base_amt','mean'),
                                 monthly_std_amount=('base_amt','std')
                                )
out_df = out_df.reset_index(level=["source"])
out_df = out_df.reset_index(level=["tran_timestamp"])

# rename some columns
out_df = out_df.rename(columns={"source": "id",
                                                  "monthly_count": "monthly_out_count",
                                                  "monthly_total_amount": "monthly_out_total_amount",
                                                  "monthly_mean_amount": "monthly_out_mean_amount",
                                                  "monthly_std_amount": "monthly_out_std_amount"}, errors="raise")

out_df.tran_timestamp = out_df.tran_timestamp.values.astype(np.int64) // 10 ** 6
out_df.head(5)

##### Incoming transactions

In [ ]:
in_df = transactions_df.groupby([pd.Grouper(key='tran_timestamp', freq='M'), 'target'])\
                            .agg(monthly_count=('target','count'), 
                                 monthly_total_amount=('base_amt','sum'),
                                 monthly_mean_amount=('base_amt','mean'),
                                 monthly_std_amount=('base_amt','std'))

in_df = in_df.reset_index(level=["target"])
in_df = in_df.reset_index(level=["tran_timestamp"])
in_df.columns  = ["tran_timestamp", "id", "monthly_in_count", "monthly_in_total_amount", "monthly_in_mean_amount", "monthly_in_std_amount"]
in_df.tran_timestamp = in_df.tran_timestamp.values.astype(np.int64) // 10 ** 6
in_df.head(5)

##### Now lets join incoming and outgoing transcations datasets

In [ ]:
in_out_df = in_df.merge(out_df, on=['tran_timestamp', 'id'], how="outer")
in_out_df =  in_out_df.fillna(0)
in_out_df.head(5)

#### Assign labels to transactions that were identified as suspicius activity

In [ ]:
alert_transactions.head(5)

In [ ]:
transaction_labels = transactions_df[["source","target","tran_id","tran_timestamp"]].merge(alert_transactions[["is_sar", "tran_id"]], on=["tran_id"], how="left")
transaction_labels.is_sar = transaction_labels.is_sar.map({True: 1, np.nan: 0})
transaction_labels.sort_values('tran_id',inplace = True)
transaction_labels.head(5)

#### Now lets prepare profile (party) dataset and assign lables whether they have been reported for suspicius activity or not 

In [ ]:
party.columns = ["id","type"]
party.type = party.type.map({"Individual": 0, "Organization": 1})

party.head(5)

In [ ]:
alert_transactions = transaction_labels[transaction_labels.is_sar ==1]
alert_transactions.head()

In [ ]:
alert_transactions = transaction_labels[transaction_labels.is_sar ==1]

alert_sources = alert_transactions[["source", "tran_timestamp"]]
alert_sources.columns = ["id", "tran_timestamp"]
alert_sources.head()
alert_targets = alert_transactions[["target", "tran_timestamp"]]
alert_targets.columns = ["id", "tran_timestamp"]

sar_party = pd.concat([alert_sources, alert_targets], ignore_index=True)

sar_party.sort_values(["id", "tran_timestamp"], ascending = [False, True])

# find a 1st occurence of sar per id
sar_party = sar_party.iloc[[sar_party.id.eq(id).idxmax() for id in sar_party['id'].value_counts().index]]
sar_party = sar_party.groupby([pd.Grouper(key='tran_timestamp', freq='M'), 'id']).agg(monthly_count=('id','count'))
sar_party = sar_party.reset_index(level=["id"])
sar_party = sar_party.reset_index(level=["tran_timestamp"])
sar_party.drop(["monthly_count"], axis=1, inplace=True)

sar_party["is_sar"] = sar_party["is_sar"] = 1
sar_party

In [ ]:
party_labels = party.merge(sar_party, on=["id"], how="left")
party_labels.is_sar = party_labels.is_sar.map({1.0: 1, np.nan: 0})
max_time_stamp = datetime.datetime.utcfromtimestamp(int(max(transaction_labels.tran_timestamp.values))/1e9)
party_labels = party_labels.fillna(max_time_stamp)

In [ ]:
party_labels.head(5)

# Neo4j

In [ ]:
from graphdatascience import GraphDataScience
import math

def convertToNumber(s):
    return int.from_bytes(s.encode(), 'little')

def convertFromNumber(n):
    return n.to_bytes(math.ceil(n.bit_length() / 8), 'little').decode()

In [ ]:
gds = GraphDataScience('bolt://localhost:7687', auth=('neo4j', 'hopsworks'))

In [ ]:
def pupulate_graph(input_df):
    # Neo4j node formatting ##################
    
    # Extract unique nodes visited
    nodes = party_labels[party_labels.id.isin(input_df.source) | (party_labels.id.isin(input_df.target))]
    nodes = nodes[['id']]    
    nodes = nodes.rename(columns={"id": "nodeId"},
                         errors="raise")
    
    # Convert node ID to a positive integer (Neo4j requirements)
    nodes['nodeId'] = [convertToNumber(nodeId) for nodeId in nodes['nodeId']]
    nodes['nodeId'] = nodes['nodeId'].astype(int)

    # Neo4j relationships formatting ##################
    
    relationships = input_df[['source', 'target', 'tran_id']]
    relationships = relationships.rename(columns={"source": "sourceNodeId",
                                                  "target": "targetNodeId"},
                                         errors="raise")

    # Convert source node ID to a positive integer (Neo4j requirements)
    relationships['sourceNodeId'] = [convertToNumber(sourceNodeId) for sourceNodeId in relationships['sourceNodeId']]
    relationships['sourceNodeId'] = relationships['sourceNodeId'].astype(int)

    # Convert target node ID to a positive integer (Neo4j requirements)
    relationships['targetNodeId'] = [convertToNumber(targetNodeId) for targetNodeId in relationships['targetNodeId']]
    relationships['targetNodeId'] = relationships['targetNodeId'].astype(int)

    ##################
    
    # Build Graph
    G = gds.graph.construct("transactions-graph", nodes, relationships)

    # Check if the number of nodes is correctly stored
    assert G.node_count() == len(nodes)

    # Compute embeddings
    graph_embdeddings_df = gds.node2vec.stream(G)
    
    G.drop()

    # Convert integer node ID back to original ID
    graph_embdeddings_df['nodeId'] = [convertFromNumber(nodeId) for nodeId in graph_embdeddings_df['nodeId']]

    return {"id": graph_embdeddings_df.nodeId.to_numpy(), "graph_embeddings": graph_embdeddings_df.embedding.to_numpy()}

In [ ]:
tqdm.pandas()
transaction_graphs_by_month = transaction_labels.groupby(pd.Grouper(key='tran_timestamp', freq='M')).progress_apply(lambda x: construct_graph(x))       

In [ ]:
timestamps = transaction_graphs_by_month.index.values
graph_embeddings = transaction_graphs_by_month.tolist()

In [ ]:
graph_embdeddings_df = pd.DataFrame()
for timestamp, graph_embedding in zip(timestamps, graph_embeddings):
    df_tmp = pd.DataFrame(graph_embedding)
    df_tmp["tran_timestamp"] = timestamp
    graph_embdeddings_df = pd.concat([graph_embdeddings_df, df_tmp])    
graph_embdeddings_df.head(5)

#### Convert date time to unix epoc milliseconds 

In [ ]:
transaction_labels.tran_timestamp = transaction_labels.tran_timestamp.values.astype(np.int64) // 10 ** 6
graph_embdeddings_df.tran_timestamp = graph_embdeddings_df.tran_timestamp.values.astype(np.int64) // 10 ** 6
party_labels.tran_timestamp = party_labels.tran_timestamp.map(lambda x: datetime.datetime.timestamp(x) * 1000)
party_labels.tran_timestamp = party_labels.tran_timestamp.values.astype(np.int64)

transaction_labels['month'] = pd.to_datetime(transaction_labels['tran_timestamp'], unit='ms').dt.month

In [ ]:
# Extract unique nodes
nodes = pd.DataFrame({'id': party_labels['id'].drop_duplicates()})

nodes = nodes.rename(columns={"id": "nodeId"},
                     errors="raise")

nodes['nodeId'] = [convertToNumber(nodeId) for nodeId in nodes['nodeId']]
nodes['nodeId'] = nodes['nodeId'].astype(int)

In [ ]:
nodes.head(5)

In [ ]:
len(nodes)

In [ ]:
# Rename transactions into relationships
relationships = transaction_labels[['source', 'target', 'tran_id']]

relationships = relationships.rename(columns={"source": "sourceNodeId",
                                              "target": "targetNodeId"},
                                     errors="raise")

relationships['sourceNodeId'] = [convertToNumber(sourceNodeId) for sourceNodeId in relationships['sourceNodeId']]
relationships['sourceNodeId'] = relationships['sourceNodeId'].astype(int)

relationships['targetNodeId'] = [convertToNumber(targetNodeId) for targetNodeId in relationships['targetNodeId']]
relationships['targetNodeId'] = relationships['targetNodeId'].astype(int)

In [ ]:
relationships.head(5)

In [ ]:
len(relationships)

In [ ]:
G = gds.graph.construct("transactions-graph", nodes, relationships)

In [ ]:
gds.graph.list()

In [ ]:
# Check if number of nodes is correctly stored
G.node_count() == len(nodes)

In [ ]:
# When the graph is no longer needed, it should be dropped to free up memory
#G.drop()

In [ ]:
graph_embdeddings_df = gds.node2vec.stream(G)

In [ ]:
graph_embdeddings_df.head()

In [ ]:
graph_embdeddings_df['nodeId'] = [convertFromNumber(nodeId) for nodeId in graph_embdeddings_df['nodeId']]

graph_embdeddings_df = graph_embdeddings_df.rename(columns={"nodeId": "id",
                                                           "embedding": "graph_embeddings"},
                                                   errors="raise")

In [ ]:
graph_embdeddings_df.head()

#### Compute time evolving graph embeddings

In [ ]:
transaction_graphs_by_month = transaction_labels.groupby(pd.Grouper(key='tran_timestamp', freq='M')).apply(lambda x: construct_graph(x))       

In [ ]:
timestamps = transaction_graphs_by_month.index.values
graph_embeddings = transaction_graphs_by_month.tolist()

In [ ]:
graph_embdeddings_df = pd.DataFrame()
for timestamp, graph_embedding in zip(timestamps, graph_embeddings):
    df_tmp = pd.DataFrame(graph_embedding)
    df_tmp["tran_timestamp"] = timestamp
    graph_embdeddings_df = pd.concat([graph_embdeddings_df, df_tmp])    
graph_embdeddings_df.head(5)

#### Convert date time to unix epoc milliseconds 

In [ ]:
transaction_labels.tran_timestamp = transaction_labels.tran_timestamp.values.astype(np.int64) // 10 ** 6
graph_embdeddings_df.tran_timestamp = graph_embdeddings_df.tran_timestamp.values.astype(np.int64) // 10 ** 6
party_labels.tran_timestamp = party_labels.tran_timestamp.map(lambda x: datetime.datetime.timestamp(x) * 1000)
party_labels.tran_timestamp = party_labels.tran_timestamp.values.astype(np.int64)

---

# 👮🏼‍♀️ Data Validation 

Before you define [feature groups](https://docs.hopsworks.ai/latest/generated/feature_group/) lets define [validation rules](https://docs.hopsworks.ai/latest/generated/feature_validation/) for features. You do expect some of the features to comply with certain *rules* or *expectations*. For example: a transacted amount must be a positive value. In the case of a transacted amount arriving as a negative value you can decide whether to stop it to `write` into a feature group and throw an error or allow it to be written but provide a warning. In the next section you will create feature store `expectations`, attach them to feature groups, and apply them to dataframes being appended to said feature group.

#### Data validation with Greate Expectations in Hopsworks
You can use GE library for validation in Hopsworks features store. 

##  <img src="../../images/icon102.png" width="18px"></img> Hopsworks feature store

The Hopsworks feature feature store library is Apache V2 licensed and available [here](https://github.com/logicalclocks/feature-store-api). The library is currently available for Python and JVM languages such as Scala and Java.
In this notebook, we are going to cover Python part.

You can find the complete documentation of the library here: 

The first step is to establish a connection with your Hopsworks feature store instance and retrieve the object that represents the feature store you'll be working with. 

> By default `connection.get_feature_store()` returns the feature store of the project we are working with. However, it accepts also a project name as parameter to select a different feature store.

In [ ]:
import hopsworks

project = hopsworks.login()

# Get the feature store handle for the project's feature store
fs = project.get_feature_store()

### 🔬 Expectations suite

In [ ]:
# Define Expectation Suite - no use of HSFS
import great_expectations as ge
from pprint import pprint
import json

expectation_suite = ge.core.ExpectationSuite(expectation_suite_name="aml_project_validations")
pprint(expectation_suite.to_json_dict(), indent=2)

In [ ]:
expectation_suite.add_expectation(
  ge.core.ExpectationConfiguration(
  expectation_type="expect_column_max_to_be_between",
  kwargs={"column": "monthly_in_count", "min_value": 0, "max_value": 10000000}) 
)

In [ ]:
pprint(expectation_suite)

---

## <span style="color:#ff5f27;"> 🪄 Register Feature Groups </span>

### Feature Groups

A `Feature Groups` is a logical grouping of features, and experience has shown, that this grouping generally originates from the features being derived from the same data source. The `Feature Group` lets you save metadata along features, which defines how the Feature Store interprets them, combines them and reproduces training datasets created from them.

Generally, the features in a feature group are engineered together in an ingestion job. However, it is possible to have additional jobs to append features to an existing feature group. Furthermore, `feature groups` provide a way of defining a namespace for features, such that you can define features with the same name multiple times, but uniquely identified by the group they are contained in.

> It is important to note that `feature groups` are not groupings of features for immediate training of Machine Learning models. Instead, to ensure reusability of features, it is possible to combine features from any number of groups into training datasets.

#### Transactions monthly aggregates feature group

In [ ]:
transactions_fg = fs.get_or_create_feature_group(
    name = "transactions_monthly_aml_fg",
    version = 1,
    primary_key = ["id"],
    partition_key = ["tran_timestamp"],   
    description = "transactions monthly aggregates features",
    event_time = 'tran_timestamp',
    online_enabled = True,
    statistics_config = {"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": False},
    expectation_suite=expectation_suite
)   

transactions_fg.insert(in_out_df)

#### Alert Transaction labels feature group

In [ ]:
transaction_labels_fg = fs.get_or_create_feature_group(
    name = "transaction_labels_aml_fg",
    version = 1,
    primary_key = ["tran_id"],
    partition_key = ["month"],         
    description = "alert transactions",
    event_time = 'tran_timestamp',    
    online_enabled = True,                                                
    statistics_config = {"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": False}
)

transaction_labels_fg.insert(transaction_labels)

#### Party feature group

In [ ]:
party_fg = fs.get_or_create_feature_group(
    name = "party_aml_fg",
    version = 1,
    primary_key = ["id"],
    description = "party fg with labels",
    event_time = 'tran_timestamp',        
    online_enabled = True,
    statistics_config = {"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": False}
)

party_fg.insert(party_labels)

#### Graph embeddings feature group

In [ ]:
graph_embeddings_fg = fs.get_or_create_feature_group(name="graph_embeddings_aml_fg",
                                       version=1,
                                       primary_key=["id"],
                                       description="node embeddings from transactions graph",
                                       event_time = 'tran_timestamp',     
                                       online_enabled=True,                                                
                                       statistics_config={"enabled": False, "histograms": False, "correlations": False, "exact_uniqueness": False}
                                       )

graph_embeddings_fg.insert(graph_embdeddings_df)

---

## <span style="color:#ff5f27;"> 👓 Exploration </span>

### Feature groups are now accessible and searchable in the UI
![fg-overview](images/fg_explore.gif)

## 📊 Statistics
We can explore feature statistics in the feature groups. If statistics was not enabled when feature group was created then this can be done by:

```python
transactions_fg = fs.get_or_create_feature_group(
    name = "transactions_monthly_fg", 
    version = 1)

transactions_fg.statistics_config = {
    "enabled": True,
    "histograms": True,
    "correlations": True
}

transactions_fg.update_statistics_config()
transactions_fg.compute_statistics()
```

![fg-stats](images/freature_group_stats.gif)

## <span style="color:#ff5f27;"> ⏭️ **Next:** Part 02 </span>
    
In the following notebook you will use feature groups to create feature viewa and training dataset.